# Battle of the Neighbourhoods (Week 2)
### Applied Data Science Capstone (IBM/Coursera)
### Author: Dany Muhajir Sjafiie

## Table of Contents:
* [Introduction/Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussions](#results)
* [Conclusion](#conclusion)

## Introduction/Business Problem <a name="introduction"></a>

## Data <a name="data"></a>

bdjaodancpoq

**Import Necessary Libraries**

In [11]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import geocoder
import folium 

import json
from pandas.io.json import json_normalize

import requests
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

import geopandas as gpd

#!pip install xlrd
#!pip install descartes

print("Import Database success!")

Import Database success!


**Import and Process Database**

In [12]:
#Import Table from http://gis.bpbd.jakarta.go.id/
url = "http://gis.bpbd.jakarta.go.id/geoserver/wfs?typename=geonode%3Adki_kelurahan&outputFormat=excel&version=1.0.0&request=GetFeature&service=WFS"
df = pd.read_excel(url)
df = df[["Kecamatan", "KEL_NAME"]]

south_jakarta_subdistrict = ["Kebayoran Baru", "Kebayoran Lama","Pesanggrahan", "Cilandak", "Pasar Minggu", "Jagakarsa", "Mampang Prapatan", "Pancoran", "Tebet", "Setia budi"]
df_all = df.loc[df["Kecamatan"].str.lower().isin([x.lower() for x in south_jakarta_subdistrict])]


df_all= df_all.sort_values(by=["Kecamatan"], ignore_index=True)
df_all.rename(columns={"Kecamatan":"DISTRICT", "KEL_NAME":"URBAN COMMUNITY"}, inplace=True)
df_all.replace({"DISTRICT" : {"SETIA BUDI" : "SETIABUDI"}}, inplace=True)
df_all.replace({"URBAN COMMUNITY" : {"SETIA BUDI" : "SETIABUDI"}}, inplace=True)

#Import Postcode table from 
url_postcode = "https://www.kotajakarta.id/kode-pos-di-kota-jakarta-selatan/"
df_postcode = pd.read_html(url_postcode)

newcolumns = ["POSTCODE","DISTRICT", "URBAN COMMUNITY", "LATITUDE", "LONGITUDE"]
df_all = df_all.reindex(columns=newcolumns, fill_value=0)

for count1,row1 in df_all.iterrows():
    for x in range(len(df_postcode)):
        pc = df_postcode[x]
        for count2,row2 in pc.iterrows():
            if df_all.loc[count1, "URBAN COMMUNITY"].lower() == pc.loc[count2, "Kelurahan"].lower():
                df_all.loc[count1, "POSTCODE"] = pc.loc[count2, "Kode Pos"]


In [13]:
for count1,row1 in df_all.iterrows():
    postcode = df_all.loc[count1, "POSTCODE"]
    urb_com = df_all.loc[count1,"URBAN COMMUNITY"]
    district = df_all.loc[count1,"DISTRICT"]

    g = geocoder.arcgis('{}, {} {}'.format(urb_com,district,postcode))
    df_all.loc[count1,"LATITUDE"] = g.lat
    df_all.loc[count1,"LONGITUDE"] = g.lng

df_all.head()

,POSTCODE,DISTRICT,URBAN COMMUNITY,LATITUDE,LONGITUDE
0,12430,CILANDAK,CILANDAK BARAT,-6.290702,106.797541
1,12410,CILANDAK,CIPETE SELATAN,-6.272805,106.804752
2,12420,CILANDAK,GANDARIA SELATAN,-6.270299,106.793017
3,12450,CILANDAK,PONDOK LABU,-6.309060,106.797427
4,12440,CILANDAK,LEBAK BULUS,-6.300973,106.779093


In [14]:
#Import geojson file from http://gis.bpbd.jakarta.go.id and process to only include south jakarta urban communities
southjkt_geo = gpd.read_file('http://gis.bpbd.jakarta.go.id/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3Adki_kelurahan&outputFormat=json&version=1.0.0&service=WFS&request=GetFeature')
southjkt_geo = southjkt_geo.loc[southjkt_geo["Kecamatan"].str.lower().isin([x.lower() for x in south_jakarta_subdistrict])]
southjkt_geo.replace({"KEL_NAME" : {"SETIA BUDI" : "SETIABUDI"}}, inplace=True)
southjkt_geo.replace({"Kecamatan" : {"SETIA BUDI" : "SETIABUDI"}}, inplace=True)

In [15]:
southjkt_geo

,id,KEL_NAME,Kecamatan,ID,geometry
5,dki_kelurahan.6,BANGKA,MAMPANG PRAPATAN,3171070001,"MULTIPOLYGON (((106.82394 -6.25398, 106.82396 ..."
15,dki_kelurahan.16,BINTARO,PESANGGRAHAN,3171040001,"MULTIPOLYGON (((106.76726 -6.25291, 106.76737 ..."
17,dki_kelurahan.18,BUKIT DURI,TEBET,3171090005,"MULTIPOLYGON (((106.85741 -6.21749, 106.85767 ..."
26,dki_kelurahan.27,CIKOKO,PANCORAN,3171080006,"MULTIPOLYGON (((106.85455 -6.24301, 106.85591 ..."
33,dki_kelurahan.34,CIGANJUR,JAGAKARSA,3171010003,"MULTIPOLYGON (((106.80758 -6.32098, 106.80758 ..."
35,dki_kelurahan.36,CILANDAK BARAT,CILANDAK,3171030003,"MULTIPOLYGON (((106.81115 -6.27931, 106.81081 ..."
36,dki_kelurahan.37,CILANDAK TIMUR,PASAR MINGGU,3171020001,"MULTIPOLYGON (((106.82064 -6.27599, 106.82058 ..."
43,dki_kelurahan.44,CIPEDAK,JAGAKARSA,3171010001,"MULTIPOLYGON (((106.80398 -6.34465, 106.80463 ..."
44,dki_kelurahan.45,CIPETE SELATAN,CILANDAK,3171030005,"MULTIPOLYGON (((106.79738 -6.26561, 106.79852 ..."
45,dki_kelurahan.46,CIPETE UTARA,KEBAYORAN BARU,3171060002,"MULTIPOLYGON (((106.81083 -6.26023, 106.81103 ..."


In [16]:
#Adjust latitude and longitude of Pesanggrahan as using geocoder gave an outlier 
df_all.loc[48,"LATITUDE"] = -6.255
df_all.loc[48, "LONGITUDE"] = 106.76

In [17]:
#Get South Jakarta Coordinate (initially geocoder was used to get coordinate then adjusted to get center of map)
g = geocoder.arcgis("Jakarta Selatan, DKI Jakarta")
latitude = -6.25
longitude = 106.80

#Map South Jakarta
map_southjkt = folium.Map(location=[latitude, longitude], zoom_start=12)


folium.Choropleth(
    geo_data=southjkt_geo,
    key_on='feature.properties.KEL_NAME',
    fill_color='deepskyblue').add_to(map_southjkt)

# add markers to map
for lat, lng, label in zip(df_all['LATITUDE'], df_all['LONGITUDE'], df_all['URBAN COMMUNITY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)  
    
map_southjkt

In [18]:
#Define Foursquare Credentials
CLIENT_ID = 'PVUDWQDMYGMZ0LTUGQ0KBGR1YDUDAQWZVAPMRH0MUR5HCMJ2' 
CLIENT_SECRET = '05SPTKSWNBY1IZRO1BD5UFA2ZPGKNJGVUQYI5JLEPILZYNXS' 
VERSION = '20201107' 
LIMIT = 100 

In [19]:
#Define Function to Explore Neighbourhoods
def getNearbyCafes(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Urban Community', 
                  'Urban Community Latitude', 
                  'Urban Community Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    coffee_categories = ["Café", "coffee shop", "Tea room", "Coffee Roaster", "Gaming Cafe", "Internet Cafe"]
    nearby_cafes = nearby_venues.loc[nearby_venues["Venue Category"].str.lower().isin([x.lower() for x in coffee_categories])]
    nearby_cafes.reset_index(drop=True,inplace=True)
    return(nearby_cafes)

In [20]:
#Use above function to return venues using toronto_data
southjkt_cafes = getNearbyCafes(names=df_all['URBAN COMMUNITY'],
                                   latitudes=df_all['LATITUDE'],
                                   longitudes=df_all['LONGITUDE']
                                  )

CILANDAK BARAT
CIPETE SELATAN
GANDARIA SELATAN
PONDOK LABU
LEBAK BULUS
LENTENG AGUNG
TANJUNG BARAT
SRENGSENG SAWAH
CIGANJUR
CIPEDAK
JAGAKARSA
MELAWAI
PULO
PETOGOGAN
GUNUNG
SENAYAN
GANDARIA UTARA
CIPETE UTARA
RAWA BARAT
SELONG
KRAMAT PELA
GROGOL UTARA
PONDOK PINANG
CIPULIR
KEBAYORAN LAMA SELATAN
KEBAYORAN LAMA UTARA
GROGOL SELATAN
PELA MAMPANG
MAMPANG PRAPATAN
TEGAL PARANG
BANGKA
KUNINGAN BARAT
KALIBATA
CIKOKO
RAWAJATI
PANCORAN
DUREN TIGA
PENGADEGAN
PEJATEN TIMUR
PEJATEN BARAT
JATI PADANG
RAGUNAN
CILANDAK TIMUR
KEBAGUSAN
PASAR MINGGU
PETUKANGAN SELATAN
BINTARO
PETUKANGAN UTARA
PESANGGRAHAN
ULUJAMI
KARET
PASAR MANGGIS
GUNTUR
SETIABUDI
KARET SEMANGGI
MENTENG ATAS
KARET KUNINGAN
KUNINGAN TIMUR
MENTENG DALAM
MANGGARAI SELATAN
MANGGARAI
TEBET BARAT
TEBET TIMUR
BUKIT DURI
KEBON BARU


In [21]:
#Check southjkt_venues database size
print(southjkt_cafes.shape)
southjkt_cafes.head(20)

(496, 7)


,Urban Community,Urban Community Latitude,Urban Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CILANDAK BARAT,-6.290702,106.797541,Bermvda,-6.291621,106.800385,Coffee Shop
1,CILANDAK BARAT,-6.290702,106.797541,Starbucks,-6.291604,106.799818,Coffee Shop
2,CILANDAK BARAT,-6.290702,106.797541,Bulaf Cafe,-6.287187,106.801288,Café
3,CILANDAK BARAT,-6.290702,106.797541,DJOURNAL COFFEE,-6.291642,106.799846,Coffee Shop
4,CILANDAK BARAT,-6.290702,106.797541,Kedai Tansu (Ketan Susu),-6.289239,106.802102,Café
5,CILANDAK BARAT,-6.290702,106.797541,Starbucks,-6.291290,106.804650,Coffee Shop
6,CILANDAK BARAT,-6.290702,106.797541,Liberica Coffee,-6.291662,106.800271,Coffee Shop
7,CILANDAK BARAT,-6.290702,106.797541,Mars Kitchen,-6.287163,106.795424,Café
8,CILANDAK BARAT,-6.290702,106.797541,Olive & Capers,-6.291675,106.800330,Café
9,CILANDAK BARAT,-6.290702,106.797541,EXCELSO,-6.291695,106.800087,Coffee Shop


In [22]:
#Get South Jakarta Coordinate (initially geocoder was used to get coordinate then adjusted to get center of map)
g = geocoder.arcgis("Jakarta Selatan, DKI Jakarta")
latitude = -6.25
longitude = 106.80

#Map South Jakarta
map_southjkt = folium.Map(location=[latitude, longitude], zoom_start=12)

#folium.Choropleth(
    #geo_data=southjkt_geo,
    #key_on='feature.properties.KEL_NAME',
    #fill_color='deepskyblue').add_to(map_southjkt)

# add markers to map
for lat, lng, label in zip(df_all['LATITUDE'], df_all['LONGITUDE'], df_all['URBAN COMMUNITY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)

for lat, lng, label in zip(southjkt_cafes['Venue Latitude'], southjkt_cafes['Venue Longitude'], southjkt_cafes['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)
  
map_southjkt

In [23]:
total_cafes=southjkt_cafes[["Urban Community","Venue"]].groupby("Urban Community").count().reset_index()
total_cafes.rename(columns = {"Venue": "Total Cafes"}, inplace=True)
total_cafes

,Urban Community,Total Cafes
0,BANGKA,15
1,BINTARO,4
2,BUKIT DURI,11
3,CIGANJUR,1
4,CIKOKO,6
5,CILANDAK BARAT,10
6,CILANDAK TIMUR,6
7,CIPETE SELATAN,12
8,CIPETE UTARA,5
9,DUREN TIGA,2


In [24]:
newcolumns = ["POSTCODE","DISTRICT", "URBAN COMMUNITY", "LATITUDE", "LONGITUDE", "TOTAL CAFES"]
df_all = df_all.reindex(columns=newcolumns, fill_value=0)

for count1,row1 in total_cafes.iterrows():
    for count2,row2 in df_all.iterrows():
        if total_cafes.loc[count1,"Urban Community"].lower() == df_all.loc[count2, "URBAN COMMUNITY"].lower():
            df_all.loc[count2,"TOTAL CAFES"] = total_cafes.loc[count1, "Total Cafes"]

df_all

,POSTCODE,DISTRICT,URBAN COMMUNITY,LATITUDE,LONGITUDE,TOTAL CAFES
0,12430,CILANDAK,CILANDAK BARAT,-6.290702,106.797541,10
1,12410,CILANDAK,CIPETE SELATAN,-6.272805,106.804752,12
2,12420,CILANDAK,GANDARIA SELATAN,-6.270299,106.793017,1
3,12450,CILANDAK,PONDOK LABU,-6.309060,106.797427,7
4,12440,CILANDAK,LEBAK BULUS,-6.300973,106.779093,4
5,12630,JAGAKARSA,LENTENG AGUNG,-6.323830,106.839080,2
6,12530,JAGAKARSA,TANJUNG BARAT,-6.305989,106.846537,4
7,12640,JAGAKARSA,SRENGSENG SAWAH,-6.347929,106.824149,0
8,12630,JAGAKARSA,CIGANJUR,-6.336752,106.808587,1
9,12630,JAGAKARSA,CIPEDAK,-6.352930,106.803080,0


In [25]:
#Get South Jakarta Coordinate (initially geocoder was used to get coordinate then adjusted to get center of map)
g = geocoder.arcgis("Jakarta Selatan, DKI Jakarta")
latitude = -6.25
longitude = 106.80

#Map South Jakarta
map_southjkt = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
    geo_data=southjkt_geo,
    data=df_all, 
    columns=["URBAN COMMUNITY", "TOTAL CAFES"],
    key_on='feature.properties.KEL_NAME',
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Number of Cafes in South Jakarta').add_to(map_southjkt)

# add markers to map
for lat, lng, urb, tot in zip(df_all['LATITUDE'], df_all['LONGITUDE'], df_all['URBAN COMMUNITY'], df_all["TOTAL CAFES"]):
    label = folium.Popup(str(urb) + "\n # OF CAFES:" + str(tot), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_southjkt)
  
map_southjkt

## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>